# Regression — California Housing (sklearn)


# Pinned installs for reproducibility in Colab
!pip -q install "pycaret>=3.0.4,<4" "pandas-datareader>=0.10.0" xgboost lightgbm catboost --upgrade

import os, sys, platform, subprocess, pandas as pd, numpy as np
import matplotlib.pyplot as plt

# Show GPU status (Colab: Runtime -> Change runtime type -> GPU)
try:
    import subprocess
    print(subprocess.check_output(["nvidia-smi"]).decode("utf-8"))
except Exception as e:
    print("No NVIDIA GPU detected (this is OK; PyCaret will fall back to CPU).")

print("PyCaret version check:")
import pycaret
import sklearn
print("pycaret", pycaret.__version__, "| sklearn", sklearn.__version__)


In [ ]:

from sklearn.datasets import fetch_california_housing
import pandas as pd
bunch = fetch_california_housing(as_frame=True)
df = bunch.frame
df.rename(columns={"MedHouseVal":"target_MedHouseVal"}, inplace=True)

from pycaret.regression import setup, compare_models, tune_model, finalize_model, plot_model, save_model, predict_model

exp = setup(
    data=df,
    target="target_MedHouseVal",
    session_id=42,
    use_gpu=True,
    silent=True
)

top = compare_models(sort="R2")
best = tune_model(top, optimize="R2")
plot_model(best, plot="residuals")
final = finalize_model(best)
save_path = save_model(final, "california_housing_regressor")
print("Saved pipeline at:", save_path)

sample = df.sample(5, random_state=7)
predict_model(final, data=sample).head()
